To only run one time at the beginning of the test (unless you want to change the current overlay) :

In [99]:
from pynq import Overlay
ol = Overlay("hardware/design_qspi.bit")

To run only if you made change in the overlay :

In [100]:
ol.download()
# ol?

We define our SPI hardware object : AxiQspi.

In [101]:
from pynq import MMIO

SPI_regSpace = ol.ip_dict['axi_quad_spi_0']
IP_BASE_ADDRESS = SPI_regSpace['phys_addr']
IP_ADDRESS_RNGE = SPI_regSpace['addr_range']
AxiQspi = MMIO(IP_BASE_ADDRESS, IP_ADDRESS_RNGE)

Open the config file

In [102]:
with open("style.css","r") as file:
        css_file = file.read()
# print(css_file)

In [103]:
import yaml
with open("config.yaml","r") as file:
        config_data = yaml.safe_load(file)
        # print(config_data)

In [104]:
%run nb/messages.ipynb

# Config setup
activate_exceptions = config_data["LAUNCH_CONFIG"]["activate_exceptions"]

# Launch config : choose to run all tests (with exceptions) or run a single test : see config.yaml
single_test_config = config_data["LAUNCH_CONFIG"]["single_test"]
all_test_config = config_data["LAUNCH_CONFIG"]["all_tests"]
if (single_test_config["bool"] == all_test_config["bool"]):
    test_config_error()

# Result file config : choose the result file extension to be html or txt : see config.yaml
activate_result_file_html = config_data["RESULT_FILE"]["html_config"]["bool"]
activate_result_file_txt = config_data["RESULT_FILE"]["txt_config"]["bool"]
activate_msg_prints = config_data["RESULT_FILE"]["print_msg"]["bool"]

In [105]:
%run nb/parse.ipynb
%run nb/translator.ipynb
%run nb/spi/spi_driver.ipynb
%run nb/spi/spi_pkg.ipynb
%run nb/messages.ipynb
%run nb/converter.ipynb
%run nb/package.ipynb

# Function which launch a test and writes in an html result file 
def init_html_result_file(test_file_name, test_file_path, css_file):
    result_file_path_html = "out/" + test_file_name + '_results.html'
    rd_data_queue = [] 
    result_file_html = open(result_file_path_html,"w")
    result_file_html_title = "Output Data in an HTML file"
    header_content = f'Results from {test_file_name}'
    html_content = initiate_html_content(result_file_html_title, css_file, header_content)
    result_file_html.write(html_content)
    return result_file_html

def end_html_result_file() -> None:
    result_file_html.write(end_html())
    result_file_html.close()

# Function which launch a test and writes in an txt result file 
def init_txt_result_file(test_file_name, test_file_path):
    result_file_path_txt = "out/" + test_file_name + '_results.txt'
    rd_data_queue = [] 
    result_file_txt = open(result_file_path_txt,"w")
    result_file_txt_title = f"Results from {test_file_name} \n \n"
    result_file_txt.write(result_file_txt_title)
    return result_file_txt

def end_txt_result_file() -> None:
    end_txt_file = "End of results for this file"
    result_file_txt.write(end_txt_file)
    result_file_txt.close()
        

import os 
#change to cnfg_master_spi
cnfg_master_spi(AxiQspi, clk_phase=0, clk_pol=0)
# ALL tests config :
if (all_test_config["bool"]) :
    all_test_files = os.listdir("./data")
    ignored_test_files = all_test_config['ignored_test_files']
    list_test_files = [f for f in all_test_files if f not in ignored_test_files]
    for test_file in list_test_files:
        test_file_name = test_file[:-4] # remove the txt extension
        test_file_path = 'data/' + test_file_name + '.txt'
        print(f"Currently sending test data from {test_file_name + '.txt'} ")
        
        if (activate_result_file_html):
            result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

        if (activate_result_file_txt):
            result_file_txt = init_txt_result_file(test_file_name, test_file_path)
        
        with open(test_file_path,"r") as test_file_opened:
            parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
            for parsed_line in parsed_file:
                command_disjunction(AxiQspi, result_file_html, result_file_txt, parsed_line, rd_data_queue)
                
        if (activate_result_file_html):
            end_html_result_file()

        if (activate_result_file_txt):
            end_txt_result_file()
                
        print(f"Finished sending test data from {test_file + '.txt'} ")
     
    
    
# SINGLE test config :
if (single_test_config["bool"]) :
    test_file_name = single_test_config['test_file'] 
    test_file_path = 'data/' + test_file_name + '.txt'
    print(f"Currently sending test data from {test_file + '.txt'} ")

    if (activate_result_file_html):
        result_file_html = init_html_result_file(test_file_name, test_file_path, css_file)

    if (activate_result_file_txt):
        result_file_txt = init_txt_result_file(test_file_name, test_file_path)
    
    with open(test_file_path,"r") as test_file_opened:
        parsed_file = run_parse_loop(test_file_opened, result_file_html, result_file_txt)
        for parsed_line in parsed_file:
             command_disjunction(AxiQspi, result_file_html, result_file_txt, parsed_line, rd_data_queue)
            
    if (activate_result_file_html):
        end_html_result_file()

    if (activate_result_file_txt):
        end_txt_result_file()
            
    print(f"Finished sending test data from {test_file + '.txt'} ")
    

Configure device
Currently sending test data from test_cmp.txt 
Finished sending test data from test_cmp.txt.txt 
Currently sending test data from test2.txt 
Finished sending test data from test2.txt.txt 
Currently sending test data from test1.txt 
Finished sending test data from test1.txt.txt 
Currently sending test data from test_bist_basic.txt 
Finished sending test data from test_bist_basic.txt.txt 
Currently sending test data from test1_onchip_clk.txt 
Finished sending test data from test1_onchip_clk.txt.txt 
Currently sending test data from test_bist_halt_example.txt 
Finished sending test data from test_bist_halt_example.txt.txt 
Currently sending test data from test3.txt 
Finished sending test data from test3.txt.txt 
Currently sending test data from test_bist_array_0_full_range.txt 
Finished sending test data from test_bist_array_0_full_range.txt.txt 
Currently sending test data from test4.txt 
Finished sending test data from test4.txt.txt 
